In [1]:
import pynq
import time
import numpy as np

NODES = 2392
NN_LIST_SIZE = 20
CHECKLIST_CAPACITY = 500
CHECKLIST_SIZE = 0;


In [2]:
# Parse Data from Stats.txt

f = open("stats.txt", 'r')

read_coords = False 
coords = []
nn = []
checklists = []
ant_pre_routes = []
actual_ant_post_routes = []
ls_sizes = []
for line in f.readlines():
    if (line.find("coordinates:") >= 0):
        read_coords = True
    elif (line.find("get_nearest_neighbors") >= 0):
        read_coords = False
        nn_node = line.rsplit()
        [nn_node.pop(0) for i in range(4)]
        nn_node = list(map(int, nn_node))
        nn_node = np.array(nn_node)
        nn.append(nn_node)
    elif (read_coords):
        coord = line.rsplit()
        coord = list(map(int, coord))
        coord = np.array(coord)
        coords.append(coord)
    elif (line.find("ls_checklist:") >= 0):
        ch = np.zeros((CHECKLIST_CAPACITY), dtype=int)
        temp_ch = line.rsplit()
        temp_ch.pop(0)
        temp_ch = list(map(int, temp_ch))
        temp_ch = np.array(temp_ch)
        CHECKLIST_SIZE = temp_ch.shape[0]
        if temp_ch.shape[0] < ch.shape[0]:
            for i in range(temp_ch.shape[0]):
                ch[i] = temp_ch[i]
        else:
            for i in range(ch.shape[0]):
                ch[i] = temp_ch[i]
        checklists.append(ch)
    elif (line.find("ant.route_:") >= 0):
        pre_route = line.rsplit()
        pre_route.pop(0)
        pre_route = list(map(int, pre_route))
        pre_route = np.array(pre_route)
        ant_pre_routes.append(pre_route)
    elif (line.find("ant.route_ after two_opt_nn:") >= 0):
        actual_post_route = line.rsplit()
        [actual_post_route.pop(0) for i in range(3)]
        actual_post_route = list(map(int, actual_post_route))
        actual_post_route = np.array(actual_post_route)
        actual_ant_post_routes.append(actual_post_route)
    elif (line.find("ls_cand_list_size_") >= 0):
        ls_cand_list_size_ = line.rsplit()
        ls_cand_list_size_.pop(0)
        ls_size = int(ls_cand_list_size_[0])
        ls_sizes.append(ls_size)

coords = np.array(coords)
nn = np.array(nn)
checklists = np.array(checklists)
ant_pre_routes = np.array(ant_pre_routes)
actual_ant_post_routes = np.array(actual_ant_post_routes)
ls_sizes = np.array(ls_sizes)
print(ls_sizes)

[20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20]


In [3]:
# print(coords)
# print(nn)
# print(ant_pre_routes)
# print(ant_post_routes)

print(coords.shape)
print(nn.shape)
print(ant_pre_routes.shape)
print(actual_ant_post_routes.shape)
print(checklists.shape) # Array of arrays too...

(2392, 2)
(2392, 20)
(20, 2392)
(20, 2392)
(20, 500)


In [4]:
bitstream = "two_opt_nn.bit"
overlay = pynq.Overlay(bitstream)

In [24]:
dut = overlay.two_opt_nn_0

coords_in = pynq.allocate((2392, 2), dtype='int32')
nn_in = pynq.allocate((2392, 20), dtype='int32')
pre_route_in = pynq.allocate((2392), dtype='int32')
checklist_in = pynq.allocate((500), dtype='int32')

first_in = 1
CHECKLIST_SIZE_IN = CHECKLIST_SIZE

# Copy Coords and NN arrays once.
for i in range(2392):
    for j in range(2):
        coords_in[i][j] = coords[i][j]

for i in range(2392):
    for j in range(20):
        nn_in[i][j] = nn[i][j]

        
# Copy ant route and checklist arrays. 
for i in range(2392):
    pre_route_in[i] = ant_pre_routes[0][i]

for i in range(500):
    checklist_in[i] = checklists[0][i]
    
coords_in.sync_to_device()
nn_in.sync_to_device()
pre_route_in.sync_to_device()
checklist_in.sync_to_device()

dut.register_map.first_in = first_in
dut.register_map.CHECKLIST_SIZE_IN = CHECKLIST_SIZE_IN

dut.register_map.coords_in_1 = coords_in.device_address & 0xFFFFFFFF
dut.register_map.coords_in_2 = coords_in.device_address >> 32

dut.register_map.neighbors_in_1 = nn_in.device_address & 0xFFFFFFFF
dut.register_map.neighbors_in_2 = nn_in.device_address >> 32

dut.register_map.route_in_1 = pre_route_in.device_address & 0xFFFFFFFF
dut.register_map.route_in_2 = pre_route_in.device_address >> 32

dut.register_map.checklist_in_1 = checklist_in.device_address & 0xFFFFFFFF
dut.register_map.checklist_in_2 = checklist_in.device_address >> 32

In [25]:
# FPGA RUN
start_time = time.time()

dut.register_map.CTRL.AP_START = 1
dut.register_map.CTRL[4] = 1
while not dut.register_map.CTRL.AP_DONE: pass

end_time = time.time()
duration = end_time - start_time
print(f'Kernel completed in {duration * 1000:.2f}ms')

print("DISTANCE CHECK")
# DISTANCE CHECK
post_route = np.zeros(2392, dtype=int)
for i in range(2392):
    post_route[i] = pre_route_in[i]

import math 
def get_distance(a, b):
    # val = math.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)
    val = ((a[0] - b[0])**2 + (a[1] - b[1])**2)**(0.5)
    # print(val)
    return val

route_len = 0
for i in range(1,2392):
    route_len += get_distance(coords[ant_pre_routes[0][i]], coords[ant_pre_routes[0][i-1]])
print("Pre Two Opt Route Distance: ", int(round(route_len)))

route_len = 0
for i in range(1,2392):
    route_len += get_distance(coords[post_route[i]], coords[post_route[i-1]])
print("Post Two Opt Route Distance: ", int(round(route_len)))

route_len = 0
for i in range(1,2392):
    route_len += get_distance(coords[actual_ant_post_routes[0][i]], coords[actual_ant_post_routes[0][i-1]])
print("Actual Two Opt Route Distance (Stats.txt): ", int(round(route_len)))




Kernel completed in 6.48ms
DISTANCE CHECK
Pre Two Opt Route Distance:  396797
Post Two Opt Route Distance:  393875
Actual Two Opt Route Distance (Stats.txt):  393875


In [28]:
# 20 Iterations 
for iteration in range(20):
    first_in = iteration == 0
    CHECKLIST_SIZE_IN = CHECKLIST_SIZE
       
    # Copy ant route and checklist arrays. 
    for i in range(2392):
        pre_route_in[i] = ant_pre_routes[iteration][i]

    for i in range(500):
        checklist_in[i] = checklists[iteration][i]

    coords_in.sync_to_device()
    nn_in.sync_to_device()
    pre_route_in.sync_to_device()
    checklist_in.sync_to_device()

    dut.register_map.first_in = first_in
    dut.register_map.CHECKLIST_SIZE_IN = CHECKLIST_SIZE_IN

    dut.register_map.coords_in_1 = coords_in.device_address & 0xFFFFFFFF
    dut.register_map.coords_in_2 = coords_in.device_address >> 32

    dut.register_map.neighbors_in_1 = nn_in.device_address & 0xFFFFFFFF
    dut.register_map.neighbors_in_2 = nn_in.device_address >> 32

    dut.register_map.route_in_1 = pre_route_in.device_address & 0xFFFFFFFF
    dut.register_map.route_in_2 = pre_route_in.device_address >> 32

    dut.register_map.checklist_in_1 = checklist_in.device_address & 0xFFFFFFFF
    dut.register_map.checklist_in_2 = checklist_in.device_address >> 32
    
    # FPGA RUN
    start_time = time.time()

    dut.register_map.CTRL.AP_START = 1
    dut.register_map.CTRL[4] = 1
    while not dut.register_map.CTRL.AP_DONE: pass

    end_time = time.time()
    duration = end_time - start_time
    print(f'Kernel completed in {duration * 1000:.2f}ms')

    print("DISTANCE CHECK")
    # DISTANCE CHECK
    post_route = np.zeros(2392, dtype=int)
    for i in range(2392):
        post_route[i] = pre_route_in[i]

    import math 
    def get_distance(a, b):
        val = ((a[0] - b[0])**2 + (a[1] - b[1])**2)**(0.5)
        return val

    route_len = 0
    for i in range(1,2392):
        route_len += get_distance(coords[ant_pre_routes[iteration][i]], coords[ant_pre_routes[iteration][i-1]])
    print("Pre Two Opt Route Distance: ", int(round(route_len)))

    route_len = 0
    for i in range(1,2392):
        route_len += get_distance(coords[post_route[i]], coords[post_route[i-1]])
    print("Post Two Opt Route Distance: ", int(round(route_len)))

    route_len = 0
    for i in range(1,2392):
        route_len += get_distance(coords[actual_ant_post_routes[iteration][i]], coords[actual_ant_post_routes[iteration][i-1]])
    print("Actual Two Opt Route Distance (Stats.txt): ", int(round(route_len)))

Kernel completed in 4.16ms
DISTANCE CHECK
Pre Two Opt Route Distance:  396797
Post Two Opt Route Distance:  393875
Actual Two Opt Route Distance (Stats.txt):  393875
Kernel completed in 1.63ms
DISTANCE CHECK
Pre Two Opt Route Distance:  400481
Post Two Opt Route Distance:  394165
Actual Two Opt Route Distance (Stats.txt):  394165
Kernel completed in 1.64ms
DISTANCE CHECK
Pre Two Opt Route Distance:  396366
Post Two Opt Route Distance:  393699
Actual Two Opt Route Distance (Stats.txt):  393699
Kernel completed in 0.65ms
DISTANCE CHECK
Pre Two Opt Route Distance:  395970
Post Two Opt Route Distance:  393866
Actual Two Opt Route Distance (Stats.txt):  393866
Kernel completed in 1.41ms
DISTANCE CHECK
Pre Two Opt Route Distance:  397136
Post Two Opt Route Distance:  393714
Actual Two Opt Route Distance (Stats.txt):  393714
Kernel completed in 1.61ms
DISTANCE CHECK
Pre Two Opt Route Distance:  397439
Post Two Opt Route Distance:  394323
Actual Two Opt Route Distance (Stats.txt):  394323
Kern